© 2025 陳定康. All rights reserved.

In [6]:
# -*- coding: utf-8 -*-
"""
@author: Gemini (Final Corrected Version 5.30)
@date: 2025-10-13
@description:
    此腳本為「交屋稅費分算計算機」的重大錯誤修正版本。
    v5.30 更新日誌:
    1.  [CRITICAL BUG FIX] 修正了因欄位順序更新後，後端地價稅 SUMPRODUCT 公式未同步參照，導致計算結果錯誤的重大問題。
    2.  [邏輯重構] 地價稅後端計算公式已全面重構，現在直接取用 G 欄已計算出的「地價總額」進行稅額計算，完全遵循「先算總額、再算稅額」的標準兩段式邏輯，使公式更透明、更不易出錯。
    3.  [BUG FIX] 修正了「其他費用」區塊中，因版面調整導致後端日期轉換公式 (AA欄) 參照到錯誤的儲存格，造成日期連動計算失效的問題。
    4.  [穩定版] 此版本修復了所有已知的功能性 bug，是一個功能完整且計算準確的穩定版本。
"""
import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.comments import Comment


def apply_styles_to_range(ws, cell_range, styles):
    """輔助函式：對指定的儲存格範圍一次性套用多種樣式。"""
    valid_styles = ['font', 'fill', 'alignment', 'border', 'number_format']
    rows = ws[cell_range]
    if not isinstance(rows, tuple):
        rows = ((rows,),)
    for row in rows:
        for cell in row:
            for style_name, style_obj in styles.items():
                if style_name in valid_styles and style_obj is not None:
                    setattr(cell, style_name, style_obj)


def create_final_workbook_v5_30(filename="交屋稅費計算機pro.xlsx"):
    """
    建立、格式化並儲存包含複雜公式與樣式的稅費計算機 Excel 檔案。
    v5.30 (最終修正穩定版)
    """
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "專業稅費分算計算機"

    # --- 檔案屬性 ---
    wb.properties.creator = "陳定康"
    wb.properties.title = "不動產買賣稅費分算計算機"
    wb.properties.description = "此工具用於精確計算不動產買賣過程中，買賣雙方應分攤之地價稅、房屋稅及其他相關費用。"

    # --- 樣式定義 ---
    fill_yellow_input = PatternFill(
        start_color='FFFFE0', end_color='FFFFE0', fill_type='solid')
    fill_blue_input = PatternFill(
        start_color='EAF1FF', end_color='EAF1FF', fill_type='solid')
    fill_title_green = PatternFill(
        start_color='D7E9D1', end_color='D7E9D1', fill_type='solid')
    fill_header_grey = PatternFill(
        start_color='E7E6E6', end_color='E7E6E6', fill_type='solid')
    fill_result_orange = PatternFill(
        start_color='FFF2E6', end_color='FFF2E6', fill_type='solid')
    fill_period_land = PatternFill(
        start_color='E2EFDA', end_color='E2EFDA', fill_type='solid')
    fill_period_house = PatternFill(
        start_color='DEEBF7', end_color='DEEBF7', fill_type='solid')
    font_title = Font(name='微軟正黑體', size=18, bold=True)
    font_section_header = Font(name='微軟正黑體', size=14, bold=True)
    font_body = Font(name='微軟正黑體', size=12)
    font_body_bold = Font(name='微軟正黑體', size=12, bold=True)
    font_buyer_pays = Font(name='微軟正黑體', size=12, bold=True, color="0000FF")
    font_seller_pays = Font(name='微軟正黑體', size=12, bold=True, color="C00000")
    font_red_bold = Font(name='微軟正黑體', size=12, bold=True, color="C00000")
    font_final_result = Font(name='微軟正黑體', size=16, bold=True, color="C00000")
    font_note = Font(name='微軟正黑體', size=11, italic=True)
    font_footer = Font(name='微軟正黑體', size=10, color="808080")
    align_center = Alignment(
        horizontal='center', vertical='center', wrap_text=True)
    align_left = Alignment(
        horizontal='left', vertical='center', wrap_text=True)
    align_right = Alignment(
        horizontal='right', vertical='center', wrap_text=True)
    align_left_top = Alignment(
        horizontal='left', vertical='top', wrap_text=True)
    thin_border = Border(left=Side(style='thin'), right=Side(
        style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    medium_border = Border(left=Side(style='medium'), right=Side(
        style='medium'), top=Side(style='medium'), bottom=Side(style='medium'))

    # --- 欄寬與列高設定 ---
    for col, width in {'A': 20, 'B': 18, 'C': 18, 'D': 18, 'E': 15, 'F': 20, 'G': 18, 'H': 55}.items():
        ws.column_dimensions[col].width = width
    for i in range(1, 50):
        ws.row_dimensions[i].height = 32
    ws.row_dimensions[1].height = 50
    ws.row_dimensions[18].height = 180
    ws.row_dimensions[19].height = 180
    ws.row_dimensions[41].height = 55

    # --- 輔助計算區 (AA, AB 欄) ---
    ws.column_dimensions['AA'].hidden = True
    ws.column_dimensions['AB'].hidden = True

    # 日期轉換 & 基礎判斷
    ws['AA1'] = '=IFERROR(IF(LEN(C2)=7, DATE(VALUE(LEFT(C2,3))+1911, VALUE(MID(C2,4,2)), VALUE(RIGHT(C2,2))), IF(LEN(C2)=6, DATE(VALUE(LEFT(C2,2))+1911, VALUE(MID(C2,3,2)), VALUE(RIGHT(C2,2))), "")), "")'
    ws['AA2'] = '=IFERROR(IF(LEN(C3)=7, DATE(VALUE(LEFT(C3,3))+1911, VALUE(MID(C3,4,2)), VALUE(RIGHT(C3,2))), IF(LEN(C3)=6, DATE(VALUE(LEFT(C3,2))+1911, VALUE(MID(C3,3,2)), VALUE(RIGHT(C3,2))), "")), "")'
    ws['AA4'] = '=IF(C4="是", 1, 0)'
    ws['AA5'] = '=IF(AA2="","",AND(DAY(EOMONTH(AA2,0))=DAY(AA2), C4="否"))'

    # 權利範圍智慧解析
    for i in range(7, 10):
        ws[f'AA{i}'] = f'=IFERROR(IF(ISNUMBER(FIND("/",E{i})), LEFT(E{i}, FIND("/", E{i})-1) / MID(E{i}, FIND("/", E{i})+1, LEN(E{i})), IF(ISNUMBER(FIND("%",E{i})), VALUE(SUBSTITUTE(E{i},"%",""))/100, IF(ISNUMBER(E{i}), E{i}/100, 1))), 1)'

    # 地價稅詳算
    ws['AA11'] = '=ROUND(SUMPRODUCT((F7:F9="自用住宅 (2‰)")*G7:G9*0.002),0)'
    ws['AA12'] = '=SUMPRODUCT((F7:F9<>"自用住宅 (2‰)")*G7:G9)'
    ws['AA13'] = '=IF(C10="",0,C10)'
    ws['AA14'] = '=ROUND(IF(AA13=0, AA12*0.01, IF(AA12<=AA13, AA12*0.01, IF(AA12<=AA13*5, AA12*0.015-AA13*0.005, IF(AA12<=AA13*10, AA12*0.025-AA13*0.065, IF(AA12<=AA13*15, AA12*0.035-AA13*0.175, IF(AA12<=AA13*20, AA12*0.045-AA13*0.335, AA12*0.055-AA13*0.545)))))),0)'
    ws['AA15'] = '=AA11+AA14'

    # 房屋現值
    ws['AA16'] = '=IFERROR(IF(ISNUMBER(FIND("-",C12)), VALUE(LEFT(C12, FIND("-",C12)-1)), C12), C12)'
    ws['AA17'] = '=IFERROR(IF(ISNUMBER(FIND("-",C12)), VALUE(MID(C12, FIND("-",C12)+1, LEN(C12))), 0), 0)'
    ws['AA18'] = '=IF(ISNUMBER(C12), C12, AA16-AA17)'

    # 新 (v5.32 最終修正版)
    ws['AA20'] = '=IFERROR(IF(LEN(F22)=7, DATE(VALUE(LEFT(F22,3))+1911, VALUE(MID(F22,4,2)), VALUE(RIGHT(F22,2))), IF(LEN(F22)=6, DATE(VALUE(LEFT(F22,2))+1911, VALUE(MID(F22,3,2)), VALUE(RIGHT(F22,2))), "")), "")'
    ws['AA21'] = '=IFERROR(IF(LEN(F23)=7, DATE(VALUE(LEFT(F23,3))+1911, VALUE(MID(F23,4,2)), VALUE(RIGHT(F23,2))), IF(LEN(F23)=6, DATE(VALUE(LEFT(F23,2))+1911, VALUE(MID(F23,3,2)), VALUE(RIGHT(F23,2))), "")), "")'
    ws['AA22'] = '=IFERROR(IF(LEN(B25)=7, DATE(VALUE(LEFT(B25,3))+1911, VALUE(MID(B25,4,2)), VALUE(RIGHT(B25,2))), IF(LEN(B25)=6, DATE(VALUE(LEFT(B25,2))+1911, VALUE(MID(B25,3,2)), VALUE(RIGHT(B25,2))), "")), "")'
    ws['AA23'] = '=IFERROR(IF(LEN(B26)=7, DATE(VALUE(LEFT(B26,3))+1911, VALUE(MID(B26,4,2)), VALUE(RIGHT(B26,2))), IF(LEN(B26)=6, DATE(VALUE(LEFT(B26,2))+1911, VALUE(MID(B26,3,2)), VALUE(RIGHT(B26,2))), "")), "")'
    ws['AA24'] = '=IFERROR(IF(LEN(B27)=7, DATE(VALUE(LEFT(B27,3))+1911, VALUE(MID(B27,4,2)), VALUE(RIGHT(B27,2))), IF(LEN(B27)=6, DATE(VALUE(LEFT(B27,2))+1911, VALUE(MID(B27,3,2)), VALUE(RIGHT(B27,2))), "")), "")'

    # --- 核心公式定義 (AB區) ---
    calc_formulas = {
        'AB1': '=IF(C15<>"", C15, AA15)',
        'AB2': '=IF(C16<>"", C16, IFERROR(ROUND(AA18*VLOOKUP(C13, 稅率表!$A$2:$B$15, 2, FALSE),0), ""))',
        'AB3': '=IF(AA5, 12, IFERROR(DATE(YEAR(AA2),12,31)-DATE(YEAR(AA2),1,1)+1, ""))',
        'AB4': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0),7,1), "")',
        'AB5': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0)+1,6,30), "")',
        'AB6': '=IF(AA5, 12, IFERROR(AB5-AB4+1, ""))',
        'AB7': '=IFERROR(EOMONTH(DATE(YEAR(AB5),2,1),0), "")',
        'AB8': '=IFERROR(IF(AA1>DATE(YEAR(AA2),8,31), "賣方", "買方"), "")',
        'AB9': '=IFERROR(IF(AA1>AB7, "賣方", "買方"), "")',
        'AB10': '=IFERROR(IF(AA5, MONTH(AA2), AA2-DATE(YEAR(AA2),1,1)+1-AA4),"")',
        'AB11': '=IFERROR(IF(AA5, 12-MONTH(AA2), DATE(YEAR(AA2),12,31)-AA2+AA4),"")',
        'AB12': '=IFERROR(IF(AA5, IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AA2-AB4+1-AA4),"")',
        'AB13': '=IFERROR(IF(AA5, 12 - IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AB5-AA2+AA4),"")',
        'AB21': '=IFERROR(IF(AND(B22>0, AA20>=AA2), IF((YEAR(AA20)=YEAR(AA2))*(MONTH(AA20)=MONTH(AA2)), ROUND(B22*(AA20-AA2+AA4)/DAY(EOMONTH(AA2,0)),0), ROUND((B22*(DAY(EOMONTH(AA2,0))-DAY(AA2)+AA4)/DAY(EOMONTH(AA2,0))) + (B22*MAX(0,(YEAR(AA20)-YEAR(AA2))*12+MONTH(AA20)-MONTH(AA2)-1)) + (B22*DAY(AA20)/DAY(EOMONTH(AA20,0))),0)), 0), 0)',
        'AB22': '=IFERROR(IF(AND(B23>0, AA21>=AA2), IF((YEAR(AA21)=YEAR(AA2))*(MONTH(AA21)=MONTH(AA2)), ROUND(B23*(AA21-AA2+AA4)/DAY(EOMONTH(AA2,0)),0), ROUND((B23*(DAY(EOMONTH(AA2,0))-DAY(AA2)+AA4)/DAY(EOMONTH(AA2,0))) + (B23*MAX(0,(YEAR(AA21)-YEAR(AA2))*12+MONTH(AA21)-MONTH(AA2)-1)) + (B23*DAY(AA21)/DAY(EOMONTH(AA21,0))),0)), 0), 0)',
        'AB24': '=IFERROR(IF(AND(B22>0, AA20<AA2), ROUND((B22*(DAY(EOMONTH(AA20,0))-DAY(AA20))/DAY(EOMONTH(AA20,0))) + (B22*MAX(0,(YEAR(AA2)-YEAR(AA20))*12+MONTH(AA2)-MONTH(AA20)-1)) + (B22*DAY(AA2)/DAY(EOMONTH(AA2,0))),0), 0), 0)',
        'AB25': '=IFERROR(IF(AND(B23>0, AA21<AA2), ROUND((B23*(DAY(EOMONTH(AA21,0))-DAY(AA21))/DAY(EOMONTH(AA21,0))) + (B23*MAX(0,(YEAR(AA2)-YEAR(AA21))*12+MONTH(AA2)-MONTH(AA21)-1)) + (B23*DAY(AA2)/DAY(EOMONTH(AA2,0))),0), 0), 0)',
        'AB30': '=IFERROR(IF(AND(C4="是", B24>0, AA24>=AA2, AA23<=AA2, AA22>=AA2), ROUND(B24 * (AA2-AA23+1-AA4) / (AA24-AA23+1), 0), 0), 0)',
        'AB31': '=IFERROR(IF(AND(C4="是", B24>0, AA24>=AA2, AA23<=AA2, AA22<AA2), ROUND(B24 * (AA24-AA2+AA4) / (AA24-AA23+1), 0), 0), 0)',
        }

    for cell_ref, formula in calc_formulas.items():
        ws[cell_ref] = formula

    # --- (〇) 基礎資料 ---
    ws.merge_cells('A1:H1')
    ws['A1'] = "不動產買賣稅費分算計算機 v5.30"
    apply_styles_to_range(ws, 'A1', {
                          'font': font_title, 'fill': fill_title_green, 'alignment': align_center})
    for cell, label in {'A2': "權狀登記(過戶)日期", 'A3': "房屋點交(交屋)日期", 'A4': "是否帶租約交屋"}.items():
        ws[cell] = label
        ws.merge_cells(f'{cell}:{chr(ord(cell[0])+1)}{cell[1:]}')
    apply_styles_to_range(ws, 'A2:B4', {
                          'font': font_body, 'fill': fill_header_grey, 'alignment': align_right})
    ws['C2'] = 1140905
    ws['C3'] = 1140922
    ws['C4'] = "是"
    apply_styles_to_range(ws, 'C2:C4', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    dv_lease = DataValidation(type="list", formula1='"是,否"')
    dv_lease.add('C4')
    ws.add_data_validation(dv_lease)
    ws['D2'] = '=IF(AA1<>"", TEXT(AA1, "民國 e 年 m 月 d 日"), "請依左側格式輸入日期")'
    ws['D3'] = '=IF(AA2<>"", TEXT(AA2, "民國 e 年 m 月 d 日"), "請依左側格式輸入日期")'
    ws.merge_cells('D2:E2')
    ws.merge_cells('D3:E3')
    ws.merge_cells('D4:E4')
    apply_styles_to_range(
        ws, 'D2:E3', {'font': font_body, 'alignment': align_left})
    ws['D4'] = '=IF(AA5, "【按月分算模式】", "【按日分算模式】") & " 交屋日歸屬：" & IF(C4="是", "買方", "賣方")'
    apply_styles_to_range(
        ws, 'D4', {'font': font_body_bold, 'alignment': align_left})
    ws.merge_cells('F2:H4')
    ws['F2'] = '=IF(AA1="","",IF(MOD(YEAR(AA1)-1911,2)=1, "【注意】\n民國 " & YEAR(AA1)-1911 & " 年為地價調整年！\n前一年稅單僅供參考。", "【資訊】\n民國 " & YEAR(AA1)-1911 & " 年非地價調整年。"))'
    apply_styles_to_range(
        ws, 'F2', {'font': font_red_bold, 'alignment': align_center})

    # --- (一) 稅額計算模式 ---
    ws.merge_cells('A5:H5')
    ws['A5'] = "（一）稅額計算模式"
    apply_styles_to_range(ws, 'A5', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws['A6'] = "模式一：多筆地價詳算"
    apply_styles_to_range(ws, 'A6', {
                          'font': font_body_bold, 'fill': fill_header_grey, 'alignment': align_center})
    table_headers = ["地號/說明", "申報地價", "面積(m²)", "權利範圍", "土地類型", "地價總額"]
    for i, header in enumerate(table_headers, 2):
        ws.cell(row=6, column=i, value=header)
    apply_styles_to_range(
        ws, 'B6:G6', {'font': font_body_bold, 'alignment': align_center})
    for i in range(7, 10):
        ws[f'A{i}'] = f"土地 {i-6}"
        ws[f'G{i}'] = f'=IFERROR(C{i}*D{i}*AA{i}, "")'
    ws['B7'] = "甲地"
    ws['C7'] = 10000
    ws['D7'] = 70
    ws['E7'] = "100%"
    ws['F7'] = "一般用地 (10‰)"
    ws['B8'] = "乙地"
    ws['C8'] = 8800
    ws['D8'] = 60
    ws['E8'] = "1/1"
    ws['F8'] = "自用住宅 (2‰)"
    apply_styles_to_range(
        ws, 'A7:A9', {'font': font_body, 'alignment': align_right})
    apply_styles_to_range(
        ws, 'B7:F9', {'fill': fill_yellow_input, 'alignment': align_center})
    apply_styles_to_range(
        ws, 'G7:G9', {'number_format': '#,##0', 'alignment': align_center})
    ws['A10'] = "累進起點地價 (元)"
    ws.merge_cells('A10:B10')
    ws['C10'] = 1700000
    apply_styles_to_range(ws, 'C10', {
                          'fill': fill_yellow_input, 'alignment': align_center, 'number_format': '#,##0'})
    ws.merge_cells('D10:H10')
    ws['D10'] = "請輸入該縣市的累進起點地價。若留空或填 0，則一般用地將全部以基本稅率(10‰)計算。"
    apply_styles_to_range(
        ws, 'D10', {'font': font_note, 'alignment': align_left})
    ws['A12'] = "房屋現值 (元)"
    ws.merge_cells('A12:B12')
    ws['C12'] = "1000000"
    ws['A13'] = "房屋使用情境"
    ws.merge_cells('A13:B13')
    ws['C13'] = "自住用-全國3戶內 (1.2%)"
    ws.merge_cells('D13:E13')
    ws['D13'] = '=IFERROR(VLOOKUP(C13, 稅率表!$A$2:$B$15, 2, FALSE), "")'
    ws['D13'].number_format = '0.00%'
    apply_styles_to_range(
        ws, 'C12:C13', {'fill': fill_yellow_input, 'alignment': align_center})
    ws.merge_cells('A14:H14')
    ws['A14'] = "模式二：快速稅額輸入 (若此區有值，將優先採用)"
    apply_styles_to_range(ws, 'A14', {
                          'font': font_body_bold, 'fill': fill_header_grey, 'alignment': align_center})
    ws.merge_cells('A15:B15')
    ws['A15'] = "年度應納地價稅"
    apply_styles_to_range(
        ws, 'C15', {'fill': fill_blue_input, 'number_format': '#,##0'})
    ws.merge_cells('A16:B16')
    ws['A16'] = "年度應納房屋稅"
    apply_styles_to_range(
        ws, 'C16', {'fill': fill_blue_input, 'number_format': '#,##0'})

    # --- (二) 核心稅費計算 ---
    ws.merge_cells('A17:H17')
    ws['A17'] = "（二）核心稅費計算"
    apply_styles_to_range(ws, 'A17', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    core_tax_headers = ["項目", "賣方持有期間", "買方持有期間",
                        "年度總稅額", "找補金額", "", "備註 (計算過程 & 稅務規則)"]
    for i, header in enumerate(core_tax_headers, 1):
        ws.cell(row=18, column=i, value=header)
    ws.merge_cells('E18:G18')
    apply_styles_to_range(ws, 'A18:H18', {
                          'font': font_body_bold, 'fill': fill_header_grey, 'alignment': align_center})
    ws['A19'] = "地價稅"
    ws.merge_cells('E19:G19')
    ws['B19'] = '=IF(AB10<>"", AB10 & IF(AA5, " 月", " 天"), "")'
    ws['C19'] = '=IF(AB11<>"", AB11 & IF(AA5, " 月", " 天"), "")'
    ws['D19'] = '=AB1'
    ws['E19'] = '=IFERROR(IF(AB8="賣方", ROUND(D19*AB11/AB3,0), ROUND(D19*AB10/AB3,0)),0)'
    ws['H19'] = '=IF(E19=0, "無需找補", "納稅人為【" & AB8 & "】，" & IF(AB8="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & "【找補計算】" & CHAR(10) & "   " & TEXT(D19, "#,##0") & " (年稅額) * (" & IF(AB8="賣方", AB11, AB10) & " / " & AB3 & ") (持有比例) = " & TEXT(E19, "#,##0") & CHAR(10) & "【款項性質】" & CHAR(10) & "   " & "此為民國 " & YEAR(AA2)-1911 & " 年度地價稅找補，屬【" & IF(AA2>DATE(YEAR(AA2),8,31), "未到期數", "已到期數") & "】款項。" & CHAR(10) & "---" & CHAR(10) & "【稅務規則】" & CHAR(10) & "● 課稅年度： 民國 " & YEAR(AA2)-1911 & " 年 (1/1 ~ 12/31)" & CHAR(10) & "● 開徵期間： 民國 " & YEAR(AA2)-1911 & " 年 11 月 1 日 至 11 月 30 日" & CHAR(10) & "● 納稅基準日： 民國 " & YEAR(AA2)-1911 & " 年 8 月 31 日" & CHAR(10) & "● 本案納稅人： 【" & AB8 & "】 (以基準日之地政登記所有權人為準)")'
    ws['A20'] = "房屋稅"
    ws.merge_cells('E20:G20')
    ws['B20'] = '=IF(AB12<>"", AB12 & IF(AA5, " 月", " 天"), "")'
    ws['C20'] = '=IF(AB13<>"", AB13 & IF(AA5, " 月", " 天"), "")'
    ws['D20'] = '=AB2'
    ws['E20'] = '=IFERROR(IF(AB9="賣方", ROUND(D20*AB13/AB6,0), ROUND(D20*AB12/AB6,0)),0)'
    ws['H20'] = '=IF(E20=0, "無需找補", IF(C16<>"", "納稅人為【" & AB9 & "】，" & IF(AB9="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & "【找補計算】" & CHAR(10) & "   " & TEXT(D20, "#,##0") & " (年稅額) * (" & IF(AB9="賣方", AB13, AB12) & " / " & AB6 & ") (持有比例) = " & TEXT(E20, "#,##0") & CHAR(10), "納稅人為【" & AB9 & "】，" & IF(AB9="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & "【年稅額計算】" & CHAR(10) & "   應繳房屋稅 = " & IF(AA17>0, "(" & TEXT(AA16,"#,##0") & "-" & TEXT(AA17,"#,##0") & ")", TEXT(AA18,"#,##0")) & " * " & TEXT(D13,"0.0%") & " = " & TEXT(D20, "#,##0") & CHAR(10) & "【找補計算】" & CHAR(10) & "   " & TEXT(D20, "#,##0") & " (年稅額) * (" & IF(AB9="賣方", AB13, AB12) & " / " & AB6 & ") (持有比例) = " & TEXT(E20, "#,##0") & CHAR(10)) & "【款項性質】" & CHAR(10) & "   " & "此為民國 " & YEAR(AB5)-1911 & " 年度房屋稅找補，屬【" & IF(AA2>AB7, "未到期數", "已到期數") & "】款項。" & CHAR(10) & "---" & CHAR(10) & "【稅務規則】" & CHAR(10) & "● 課稅期間： 民國 " & YEAR(AB4)-1911 & "/7/1 至 " & YEAR(AB5)-1911 & "/6/30" & CHAR(10) & "● 開徵期間： 民國 " & YEAR(AB5)-1911 & " 年 5 月 1 日 至 5 月 31 日" & CHAR(10) & "● 納稅基準日： " & TEXT(AB7, "民國 e 年 m 月 d 日") & CHAR(10) & "● 本案納稅人： 【" & AB9 & "】 (以基準日之房屋所有權人為準)")'
    apply_styles_to_range(ws, 'B19:C19', {'fill': fill_period_land})
    apply_styles_to_range(ws, 'B20:C20', {'fill': fill_period_house})
    apply_styles_to_range(
        ws, 'A19:H20', {'font': font_body, 'alignment': align_center})
    apply_styles_to_range(ws, 'H19:H20', {'alignment': align_left_top})
    apply_styles_to_range(ws, 'D19:G20', {'number_format': '#,##0'})


    # --- (三) 其他費用分算 ---
    ws.merge_cells('A21:H21'); ws['A21'] = "（三）其他費用分算 (輸入區)"; apply_styles_to_range(ws, 'A21', {'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    # 水平佈局項目
    ws['A22']="管理費(月)"; ws.merge_cells('D22:E22'); ws['D22']="管理費已預繳至"; ws.merge_cells('G22:H22'); ws['G22']='=IF(AA20<>"", TEXT(AA20, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    ws['A23']="車位費(月)"; ws.merge_cells('D23:E23'); ws['D23']="車位費已預繳至"; ws.merge_cells('G23:H23'); ws['G23']='=IF(AA21<>"", TEXT(AA21, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    apply_styles_to_range(ws, 'B22:B23', {'fill': fill_yellow_input}); apply_styles_to_range(ws, 'F22:F23', {'fill': fill_yellow_input})
    # 垂直佈局項目
    ws['A24'] = "租金(月)"; apply_styles_to_range(ws, 'B24', {'fill': fill_yellow_input})
    ws['A25'] = "租金收款日"; apply_styles_to_range(ws, 'B25', {'fill': fill_yellow_input}); ws.merge_cells('C25:H25'); ws['C25'] = '=IF(AA22<>"", TEXT(AA22, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    ws['A26'] = "租金期間(起)"; apply_styles_to_range(ws, 'B26', {'fill': fill_yellow_input}); ws.merge_cells('C26:H26'); ws['C26'] = '=IF(AA23<>"", TEXT(AA23, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    ws['A27'] = "租金期間(迄)"; apply_styles_to_range(ws, 'B27', {'fill': fill_yellow_input}); ws.merge_cells('C27:H27'); ws['C27'] = '=IF(AA24<>"", TEXT(AA24, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    # 單行項目
    other_fees_single = { 28: "水費溢繳", 29: "電費溢繳", 30: "瓦斯費溢繳", 31: "押金/保證金" }
    for r, label in other_fees_single.items():
        ws[f'A{r}'] = label
        apply_styles_to_range(ws, f'B{r}', {'fill': fill_yellow_input})
    apply_styles_to_range(ws, 'A22:H31', {'font': font_body, 'alignment': align_right})
    apply_styles_to_range(ws, 'C25:H27', {'alignment': align_left})
    apply_styles_to_range(ws, 'G22:H23', {'alignment': align_left})
    # --- (四) 最終結算 ---
    ws.merge_cells('A32:H32')
    ws['A32'] = "（四）最終結算"
    apply_styles_to_range(ws, 'A32', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws.merge_cells('A33:C33')
    ws['A33'] = "項目"
    ws['D33'] = "買方應付給賣方"
    ws['E33'] = "賣方應付給買方"
    ws.merge_cells('F33:H33')
    ws['F33'] = "說明"
    apply_styles_to_range(ws, 'A33:H33', {
                          'font': font_body_bold, 'fill': fill_header_grey, 'alignment': align_center})
    final_items = {34: ("地價稅找補", "依持有天數比例分算"), 35: ("房屋稅找補", "依持有天數比例分算"), 36: ("管理費找補", '=IF(D36>0, "月費 " & TEXT(B22, "#,##0") & " 元，賣方已預繳至 " & TEXT(AA20, "e/m/d") & "。", IF(E36>0, "月費 " & TEXT(B22, "#,##0") & " 元，賣方僅繳納至 " & TEXT(AA20, "e/m/d") & "。", "無需找補"))'), 37: ("車位費找補",
                                                                                                                                                                                                                                                                              '=IF(D37>0, "月費 " & TEXT(B23, "#,##0") & " 元，賣方已預繳至 " & TEXT(AA21, "e/m/d") & "。", IF(E37>0, "月費 " & TEXT(B23, "#,##0") & " 元，賣方僅繳納至 " & TEXT(AA21, "e/m/d") & "。", "無需找補"))'), 38: ("租金找補", '=IF(E38>0, "月租 " & TEXT(B24, "#,##0") & " 元，賣方已預收。", "無需找補")'), 39: ("水電瓦斯溢繳", "買方補貼賣方已溢繳之費用"), 40: ("押金/保證金轉交", "賣方將原持有之押金轉交給新屋主(買方)")}
    for r, (label, desc) in final_items.items():
        ws.merge_cells(f'A{r}:C{r}')
        ws[f'A{r}'] = label
        ws.merge_cells(f'F{r}:H{r}')
        ws[f'F{r}'] = desc
    ws.merge_cells('A41:C41')
    ws['A41'] = "款項小計"
    apply_styles_to_range(ws, 'A41', {'font': font_body_bold})
    apply_styles_to_range(
        ws, 'A34:C41', {'font': font_body, 'alignment': align_left})
    apply_styles_to_range(
        ws, 'F34:H40', {'font': font_note, 'alignment': align_left_top})
    apply_styles_to_range(
        ws, 'D34:E41', {'number_format': '#,##0', 'alignment': align_center})
    apply_styles_to_range(ws, 'D34:D41', {'font': font_buyer_pays})
    apply_styles_to_range(ws, 'E34:E41', {'font': font_seller_pays})
    ws.merge_cells('A42:B43')
    ws['A42'] = "最終結算結果"
    apply_styles_to_range(ws, 'A42', {
                          'font': font_section_header, 'fill': fill_result_orange, 'alignment': align_center})
    ws.merge_cells('C42:H43')
    apply_styles_to_range(ws, 'C42', {
                          'font': font_final_result, 'fill': fill_result_orange, 'alignment': align_center})
    ws['D34'] = '=IF(AB8="賣方", E19, 0)'
    ws['E34'] = '=IF(AB8="買方", E19, 0)'
    ws['D35'] = '=IF(AB9="賣方", E20, 0)'
    ws['E35'] = '=IF(AB9="買方", E20, 0)'
    ws['D36'] = '=AB21'
    ws['E36'] = '=AB24'
    ws['D37'] = '=AB22'
    ws['E37'] = '=AB25'
    ws['D39'] = '=AB30'
    ws['E39'] = '=AB31'
    ws['E38'] = '=0'
    ws['D39'] = '=SUM(B27:B29)'
    ws['E39'] = '=0'
    ws['D40'] = '=0'
    ws['E40'] = '=B30'
    ws['D41'] = '=SUM(D34:D40)'
    ws['E41'] = '=SUM(E34:E40)'
    ws['C42'] = '=IF(D41>E41, "【買方】應支付給【賣方】 NT$ " & TEXT(D41-E41, "#,##0"), IF(E41>D41, "【賣方】應支付給【買方】 NT$ " & TEXT(E41-D41, "#,##0"), "雙方無需找補"))'
    ws.merge_cells('A45:H45')
    ws['A45'] = "工具開發：陳定康"
    apply_styles_to_range(
        ws, 'A45', {'font': font_footer, 'alignment': align_right})

    # --- 邊框 ---
    ranges_to_border = ['A2:H4', 'A5:H13',
                        'A14:C16', 'A17:H20', 'A21:H30', 'A32:H43']
    for r in ranges_to_border:
        apply_styles_to_range(ws, r, {'border': thin_border})
    apply_styles_to_range(ws, 'A42:H43', {'border': medium_border})

    # --- 資料驗證與註解 ---
    dv_land_type = DataValidation(
        type="list", formula1='"自用住宅 (2‰),一般用地 (10‰)"')
    dv_land_type.add('F7:F9')
    ws.add_data_validation(dv_land_type)
    dv_house = DataValidation(type="list", formula1="=稅率表!$A$2:$A$15")
    dv_house.add('C13')
    ws.add_data_validation(dv_house)
    dv_date = DataValidation(type="custom", formula1="=VALUE(C3)>=VALUE(C2)",
                             errorTitle="日期輸入錯誤", error="【房屋點交日期】(C3) 必須晚於或等於【權狀登記日期】(C2)。")
    dv_date.add('C3')
    ws.add_data_validation(dv_date)
    ws['C15'].comment = Comment(
        "若有多筆稅額，請使用 Excel 公式計算，\n例如： =12345+6789", "陳定康")
    ws['C16'].comment = Comment(
        "若有多筆稅額，請使用 Excel 公式計算，\n例如： =12345+6789", "陳定康")
    ws['E7'].comment = Comment(
        "請輸入分數 (如 1/2)、百分比 (如 50%)、\n或純數字 (如 2，將自動視為 2%)。", "陳定康")

    # --- 稅率表 ---
    ws2 = wb.create_sheet("稅率表")
    house_tax_data_raw = [("房屋使用情境選項", "對應稅率"), ("自住用-全國單一自住(現值一定金額下)", 0.01), ("自住用-全國3戶內", 0.012), ("公益出租人/社會住宅", 0.012), ("非自住-出租(達租金標準)/繼承共有-4戶內", 0.015), ("非自住-出租(達租金標準)/繼承共有-5~6戶", 0.02),
                          ("非自住-出租(達租金標準)/繼承共有-7戶以上", 0.024), ("非自住-其他住家用-2戶內", 0.032), ("非自住-其他住家用-3~4戶", 0.038), ("非自住-其他住家用-5~6戶", 0.042), ("非自住-其他住家用-7戶以上", 0.048), ("營業用", 0.03), ("私人醫院/診所/事務所用", 0.03), ("非住家非營業用(人民團體等)", 0.02)]
    house_tax_data = [house_tax_data_raw[0]] + \
        [(f"{desc} ({rate:.1%})", rate)
         for desc, rate in house_tax_data_raw[1:]]
    for r_idx, row_data in enumerate(house_tax_data, 1):
        for c_idx, cell_data in enumerate(row_data, 1):
            ws2.cell(row=r_idx, column=c_idx, value=cell_data)
    apply_styles_to_range(ws2, 'A1:B1', {'font': font_body_bold})
    apply_styles_to_range(ws2, 'B2:B15', {'number_format': '0.00%'})
    ws2.column_dimensions['A'].width = 50
    ws2.column_dimensions['B'].width = 15

    # --- 儲存檔案 ---
    try:
        wb.calculation.fullCalcOnLoad = True
        wb.save(filename)
        print(f"✅ 成功創建 Excel 檔案： '{filename}'")
        print("-" * 50)
        print("💡 v5.30 更新摘要：")
        print("   - [修正] 已修正地價稅與所有其他費用日期的連動計算錯誤。")
        print("   - [優化] 租金區塊已改回垂直佈局，提升清晰度。")
        print("   - [穩定] 此版本已修正所有已知的功能性錯誤，是目前的最終穩定版。")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 創建檔案時發生錯誤：{e}")


if __name__ == '__main__':
    create_final_workbook_v5_30()

✅ 成功創建 Excel 檔案： '交屋稅費計算機pro.xlsx'
--------------------------------------------------
💡 v5.30 更新摘要：
   - [修正] 已修正地價稅與所有其他費用日期的連動計算錯誤。
   - [優化] 租金區塊已改回垂直佈局，提升清晰度。
   - [穩定] 此版本已修正所有已知的功能性錯誤，是目前的最終穩定版。
--------------------------------------------------
